In [1]:
import flowshape as fs
import igl
import numpy as np
import meshplot as mp
import os
from src.utilities.fin_shape_utils import fit_fin_hull, upsample_fin_point_cloud, plot_mesh
from src.utilities.fin_class_def import FinData
from src.utilities.functions import path_leaf
import glob2 as glob

### Load fin data

In [2]:
# root = "/Users/nick/Cole Trapnell's Lab Dropbox/Nick Lammers/Nick/pecfin_dynamics/"
root = "/media/nick/hdd02/Cole Trapnell's Lab Dropbox/Nick Lammers/Nick/pecfin_dynamics/"
fin_object_path = os.path.join(root, "point_cloud_data", "fin_objects", "")
fin_object_list = sorted(glob.glob(fin_object_path + "*.pkl"))

file_ind01 = 146
seg_type = "tissue_only_best_model_tissue"
fp01 = fin_object_list[file_ind01]
point_prefix01 = path_leaf(fp01).replace("_fin_object.pkl", "")
print(point_prefix01)

fin_object = FinData(data_root=root, name=point_prefix01, tissue_seg_model=seg_type)

20240711_02_well0053_time0000


### Calculate distance from each fin nucleus to the yolk surface

In [23]:
from sklearn.metrics import pairwise_distances

full_df = fin_object.full_point_data
fin_df = full_df.loc[full_df["fin_label_curr"] == 1, :]

# orient to biological axes
fin_axis_df = fin_object.axis_fin
fin_axes = fin_object.calculate_axis_array(fin_axis_df)

shift_ref_vec = np.mean(fin_points, axis=0)

fin_points_pca = np.matmul(fin_points - shift_ref_vec, fin_axes.T)
fin_df.loc[:, ["XP", "YP", "ZP"]] = fin_points_pca

# Use simple numerical procedure to calculate distance of each fin point to the yolk
fin_points = fin_df[["X", "Y", "Z"]].to_numpy()

params = fin_object.yolk_surf_params

x_min, y_min = fin_points[:, 0].min(), fin_points[:, 1].min()
x_max, y_max = fin_points[:, 0].max(), fin_points[:, 1].max()

# Create a mesh grid for x and y values
x_vals = np.linspace(x_min, x_max, 100)
y_vals = np.linspace(y_min, y_max, 100)
X, Y = np.meshgrid(x_vals, y_vals)

yolk_xyz = np.reshape(fin_object.polyval2d(np.c_[X.ravel(), Y.ravel()], params).ravel(), (-1, 3))

dist_array = pairwise_distances(fin_points, yolk_xyz)
yolk_dist = np.min(dist_array, axis=1)
min_i = np.argmin(dist_array, axis=1)
yolk_signs = np.sign(fin_points[:, 2] - yolk_xyz[min_i, 2])
yolk_dist = np.multiply(yolk_dist, yolk_signs)

fin_df["yolk_dist"] = yolk_dist

/tmp/ipykernel_1275962/3626379287.py:36: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### Calculate fin dimensions at the base and find centerpoint 

In [26]:
# get points near surface
yolk_thresh = 5
base_fin_points = fin_df.loc[np.abs(fin_df["yolk_dist"])<=yolk_thresh, ["XP", "YP", "ZP"]].to_numpy()
base_fin_points_raw = fin_df.loc[np.abs(fin_df["yolk_dist"])<=yolk_thresh, ["X", "Y", "Z"]].to_numpy()

# calculate axis dims. Main one we care about is the AP axis ("YP")
axis_len_vec = np.max(base_fin_points, axis=0) - np.min(base_fin_points, axis=0)

# find centroid
point_center = np.mean(base_fin_points_raw, axis=0)
surf_center_i = np.argmin(np.sqrt(np.sum((yolk_xyz-point_center)**2, axis=1)))
surf_center = yolk_xyz[surf_center_i, :] # this is the one we will use

# define a local DV direction that is the cross product of the surface normal and the AP axis
surf_normal_raw, _ = fin_object.calculate_tangent_plane(fin_object.yolk_surf_params, surf_center)
if surf_normal_raw[2] > 0:
    surf_normal_raw = -surf_normal_raw

# convert the normal vector to the biological axis space
surf_normal = np.matmul(np.reshape(surf_normal_raw, (1, 3)) , fin_axes.T)[0]
surf_normal = surf_normal / np.linalg.norm(surf_normal)

# calculate local DV
dv_vec_base = np.cross(surf_normal, np.asarray([0, 1, 0]))
dv_vec_base = dv_vec_base / np.linalg.norm(dv_vec_base)

# finally, calculate local DV axis dims
dv_vec_loc = np.sum(np.multiply(dv_vec_base[np.newaxis, :], base_fin_points), axis=1)

# get axis lengths
ap_axis_len = axis_len_vec[1]
dv_axis_len = np.max(dv_vec_loc) - np.min(dv_vec_loc)
print(ap_axis_len)
print(dv_axis_len)

103.69026305359134
80.73156512960355


In [20]:
# import plotly.express as px 
# test = np.sum(np.multiply(dv_vec_base[np.newaxis, :], fin_points_pca), axis=1)
# fig = px.scatter_3d(x=fin_points_pca[:, 0], y=fin_points_pca[:, 1], z=fin_points_pca[:, 2], color=test)
# fig.show()

### Load and filter fin+yolk nuclei

In [25]:
# shift centerpoint into the oriented frame of reference
surf_center_o = np.matmul(surf_center - shift_ref_vec, fin_axes.T)

# shift fin+yolk dataset to oriented frame of reference
fin_yolk_df = full_df.loc[np.isin(full_df["fin_label_curr"], [1, 2]), :]
fin_yolk_points = fin_yolk_df[["X", "Y", "Z"]].to_numpy()
fin_yolk_points_o = np.matmul(fin_yolk_points - shift_ref_vec, fin_axes.T)

### Use AP and DV dims to capture ellipsoidal "cap" at fin base

In [68]:
# calculate yolk distances (again) and use to filter for
params = fin_object.yolk_surf_params

x_min, y_min = fin_yolk_points[:, 0].min(), fin_yolk_points[:, 1].min()
x_max, y_max = fin_yolk_points[:, 0].max(), fin_yolk_points[:, 1].max()

# Create a mesh grid for x and y values
x_vals = np.linspace(x_min, x_max, 250)
y_vals = np.linspace(y_min, y_max, 250)
X, Y = np.meshgrid(x_vals, y_vals)

yolk_xyz2 = np.reshape(fin_object.polyval2d(np.c_[X.ravel(), Y.ravel()], params).ravel(), (-1, 3))

# get nearest neighbor distances
dist_array2 = pairwise_distances(fin_yolk_points, yolk_xyz2)
yolk_dist2 = np.min(dist_array2, axis=1)
min_i2 = np.argmin(dist_array2, axis=1)
yolk_signs2 = np.sign(fin_yolk_points[:, 2] - yolk_xyz2[min_i2, 2])
yolk_dist2 = -np.multiply(yolk_dist2, yolk_signs2)

fin_yolk_df["yolk_dist"] = yolk_dist2

/tmp/ipykernel_1275962/1545269934.py:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [70]:
# get points within ellipsoidal boundary
depth_semi_axis = 25
ap_min = 50
dv_min = 30
ap_semi_axis = np.max([ap_axis_len, ap_min]) / 2
dv_semi_axis = np.max([dv_axis_len, dv_min]) / 2

# below the surface
fy_candidate_points = fin_yolk_points_o[yolk_dist2<0, :]

# (x/a)^2 + (y/b)^2 + (z/c)^2 <= 1

# AP piece is trivial
ap = ((fy_candidate_points[:, 1]-surf_center_o[1]) / ap_semi_axis)**2

# DV piece 
# dv_center = np.sum(np.dot(surf_center_o, dv_vec_base))
dv_dist = np.sum(np.multiply(fy_candidate_points-surf_center_o, dv_vec_base[np.newaxis,:]), axis=1)
dv = ((dv_dist) / dv_semi_axis)**2

# depth piece 
# depth_center = np.sum(np.dot(surf_center_o, surf_normal))
depth_dist = np.sum(np.multiply(fy_candidate_points-surf_center_o, surf_normal[np.newaxis,:]), axis=1)
dd = ((depth_dist) / depth_semi_axis)**2

# put it all together to get cap points
cap_flag = ((ap + dv + dd) <= 1)# & (depth_dist<0)

print(dv_center)
print(np.max(depth_dist))
print(np.min(depth_dist))

18.283200557615388
-0.6485893196676011
-108.62412658939249


In [71]:
np.sum(cap_flag)

98

In [73]:
import plotly.graph_objects as go
# test = np.sum(np.multiply(dv_vec_base[np.newaxis, :], fin_points_pca), axis=1)
fig = go.Figure() #px.scatter_3d
fig.add_trace(go.Scatter3d(x=fy_candidate_points[:, 0], y=fy_candidate_points[:, 1], z=fy_candidate_points[:, 2], mode="markers",
                           opacity=0.1))
fig.add_trace(go.Scatter3d(x=fy_candidate_points[cap_flag, 0], y=fy_candidate_points[cap_flag, 1],
                           z=fy_candidate_points[cap_flag, 2], opacity=0.7, mode="markers"))
# fig.add_trace(go.Scatter3d(x=yolk_xyz2[:, 0], y=yolk_xyz2[:, 1], z=yolk_xyz2[:, 2], mode="markers"))
# fig.add_trace(go.Scatter3d(x=fin_yolk_points[:, 0], y=fin_yolk_points[:, 1], z=fin_yolk_points[:, 2], mode="markers"))
fig.show()

### Load, filter, and orient nucleus centroid point cloud

In [ ]:
full_df = fin_object.full_point_data

# get just fin points (will use for orientation and dim estimation)
fin_df = full_df.loc[full_df["fin_label_curr"] == 1, :]
fin_points = fin_df[["X", "Y", "Z"]].to_numpy()

# fin + yolk
fy_df = full_df.loc[np.isin(full_df["fin_label_curr"], [1, 2], :]
fy_points = fy_df[["X", "Y", "Z"]].to_numpy()

# orient to biological axes
fin_axis_df = fin_object.axis_fin
fin_axes = fin_object.calculate_axis_array(fin_axis_df)
fin_points_pca = np.matmul(fin_points - np.mean(fin_points, axis=0), fin_axes.T)
fin_df.loc[:, ["XP", "YP", "ZP"]] = fin_points_pca

In [3]:
# sample nucleus boundary points from nucleus masks
fin_df_upsamp = upsample_fin_point_cloud(fin_object, sample_res_um=0.4, root=root, points_per_nucleus=100)

/home/nick/projects/pecfin_dynamics/src/utilities/fin_shape_utils.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fin_df["nn_scale_um"] = nn_dist_mean
100%|██████████| 597/597 [00:01<00:00, 376.20it/s]


In [4]:
import open3d as o3d

# get raw points
fin_points = fin_df_upsamp[["XP", "YP", "ZP"]].to_numpy()

# convert to point cloud format
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(fin_points)

# resample points to be more spatially uniform
min_distance = 0.5
print("Downsampling...")
sampled_points = pcd.voxel_down_sample(voxel_size=min_distance) 
print("Done")

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Downsampling...
Done


In [5]:
# import plotly.express as px
# fin_points_u = np.asarray(sampled_points.points)

# fig = px.scatter_3d(x=fin_points_u[:, 0], y=fin_points_u[:, 1], z=fin_points_u[:, 2])
# fig.update_traces(marker=dict(size=3))
# fig.show()

In [7]:
# fit a mesh 
fin_points_u = np.asarray(sampled_points.points)
fin_hull, raw_hull, wt_flag = fit_fin_hull(fin_points_u, alpha=25, n_faces=5000)
print(wt_flag)

True


In [39]:
raw_hull.faces.shape

(10118, 3)

In [8]:
#igl.read_triangle_mesh("/home/nick/projects/flowshape/demo/ABal.obj")
v, f = v, f = fin_hull.vertices.copy(), fin_hull.faces.copy()
mp.plot(v, f, shading = {"wireframe":True})

/home/nick/miniconda3/envs/flowshape_env/lib/python3.10/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float32" does not match required type "float64". A coerced copy has been created.
  warnings.warn(
/home/nick/miniconda3/envs/flowshape_env/lib/python3.10/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "uint32" does not match required type "float64". A coerced copy has been created.
  warnings.warn(


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-2.368526…

### Run spherical mapping

In [9]:
# normalize the scaling of the mesh
v = fs.normalize(v)

# run the spherical mapping flow and mobius centering
sv = fs.sphere_map(v, f)

# Now we have a spherical mesh
mp.plot(sv, f, shading = {"wireframe":True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.000453…

### Calculate the mean curvature

In [10]:
rho = fs.curvature_function(v, sv, f)

mp.plot(v,f, rho )

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0033084…

## SH decomposition

In [11]:
# this utility does the above steps + SH decomposition
# Here, using maximum degree 24
weights, Y_mat, vs = fs.do_mapping(v,f, l_max = 24)

In [12]:
# This is the array of SH weights
np.set_printoptions(threshold = 100)
print(weights)

[ 3.1985965  -0.14189246 -0.09633128 ...  0.18525062 -0.10537151
  0.13012568]


In [13]:
# Y_mat is a matrix used to convert between weights and mesh function
rho2 = Y_mat.dot(weights)
mp.plot(sv,f, c = rho2)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.000453…

### Mesh reconstruction

In [14]:
rec2 = fs.reconstruct_shape(sv, f, rho2 )
mp.plot(rec2,f, c = rho2)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.050321…

### Test with lower frequencies only

In [18]:
weights, Y_mat, vs = fs.do_mapping(v,f, l_max=4)
rec_8 = fs.reconstruct_shape(sv, f, Y_mat.dot(weights) )
mp.plot(rec_8, f, c = rho2)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.078880…